In [21]:
import pandas as pd
import time
import numpy as np
from numba import jit
from sklearn.model_selection import train_test_split
from multiprocessing import *
from sklearn.model_selection import KFold
import gc
import warnings
import lightgbm as lgb
warnings.filterwarnings("ignore")
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import xgboost as xgb


In [22]:
start = time.time()
train = pd.read_csv("/home/saurabhg/PuertoSergo/train.csv")
test = pd.read_csv("/home/saurabhg/PuertoSergo/test.csv")
print(time.time()-start)
 
y = train['target']
testid= test['id'].values

train.drop(['id','target'],axis=1,inplace=True)
test.drop(['id'],axis=1,inplace=True)

8.37518191338


In [23]:
### Drop calc
unwanted = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(unwanted, axis=1)  
test = test.drop(unwanted, axis=1)

### Great Recovery from Pascal's materpiece
### Great Recovery from Pascal's materpiece
### Great Recovery from Pascal's materpiece
### Great Recovery from Pascal's materpiece
### Great Recovery from Pascal's materpiece
#train['ps_reg_03_square'] = train['ps_reg_03']*train['ps_reg_03']
#test['ps_reg_03_square'] = test['ps_reg_03']*test['ps_reg_03']
#train['ps_car_14_square'] = train['ps_car_14']*train['ps_car_14']
#test['ps_car_14_square'] = test['ps_car_14']*test['ps_car_14']
def recon(reg):
    integer = int(np.round((40*reg)**2)) 
    for a in range(32):
        if (integer - a) % 31 == 0:
            A = a
    M = (integer - A)//31
    return A, M
train['ps_reg_A'] = train['ps_reg_03'].apply(lambda x: recon(x)[0])
train['ps_reg_M'] = train['ps_reg_03'].apply(lambda x: recon(x)[1])
train['ps_reg_A'].replace(19,-1, inplace=True)
train['ps_reg_M'].replace(51,-1, inplace=True)
test['ps_reg_A'] = test['ps_reg_03'].apply(lambda x: recon(x)[0])
test['ps_reg_M'] = test['ps_reg_03'].apply(lambda x: recon(x)[1])
test['ps_reg_A'].replace(19,-1, inplace=True)
test['ps_reg_M'].replace(51,-1, inplace=True)


### Froza's baseline
### Froza's baseline
### Froza's baseline
### Froza's baseline

d_median = train.median(axis=0)
d_mean = train.mean(axis=0)
d_skew = train.skew(axis=0)
one_hot = {c: list(train[c].unique()) for c in train.columns if c not in ['id','target']}

def transform_df(df):
    df = pd.DataFrame(df)
    dcol = [c for c in df.columns if c not in ['id','target']]
    df['ps_car_13_x_ps_reg_03'] = df['ps_car_13'] * df['ps_reg_03']
    df['negative_one_vals'] = np.sum((df[dcol]==-1).values, axis=1)
    for c in dcol:
        if '_bin' not in c: #standard arithmetic
            df[c+str('_median_range')] = (df[c].values > d_median[c]).astype(np.int)
            df[c+str('_mean_range')] = (df[c].values > d_mean[c]).astype(np.int)

    for c in one_hot:
        if len(one_hot[c])>2 and len(one_hot[c]) < 8:
            for val in one_hot[c]:
                df[c+'_oh_' + str(val)] = (df[c].values == val).astype(np.int)
    return df

def multi_transform(df):
    print('Init Shape: ', df.shape)
    p = Pool(cpu_count())
    df = p.map(transform_df, np.array_split(df, cpu_count()))
    df = pd.concat(df, axis=0, ignore_index=True).reset_index(drop=True)
    p.close(); p.join()
    print('After Shape: ', df.shape)
    return df

train = multi_transform(train)
test = multi_transform(test)





('Init Shape: ', (595212, 39))
('After Shape: ', (595212, 136))
('Init Shape: ', (892816, 39))
('After Shape: ', (892816, 136))


In [24]:
# Compute gini

# from CPMP's kernel https://www.kaggle.com/cpmpml/extremely-fast-gini-computation

@jit
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)

def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_lgb(act, preds):
#    labels = dtrain.get_label()
    gini_score = gini_normalized(act, preds)
    return 'gini', gini_score,True

In [11]:
MAX_ROUNDS = 5000
OPTIMIZE_ROUNDS = True
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50  
f_cats = [f for f in train.columns if "_cat" in f]
# Note: I set EARLY_STOPPING_ROUNDS high so that (when OPTIMIZE_ROUNDS is set)
#       I will get lots of information to make my own judgment.  You should probably
#       reduce EARLY_STOPPING_ROUNDS if you want to do actual early stopping.

In [12]:
n_splits = 5
n_estimators = 1000
folds = KFold(n_splits=n_splits, shuffle=True, random_state=1) 
imp_df = np.zeros((len(train.columns), n_splits))
oof = np.empty(len(train))
sub_preds = np.zeros((len(test),n_splits))
increase = False
np.random.seed(0)
params = {'eta': 0.025, 'max_depth': 4, 
          'subsample': 0.9, 'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':100,
          'alpha':10,
          'objective': 'binary:logistic', 'eval_metric': 'auc', 'seed': 99, 'silent': True}

In [25]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(y, y)):
    trn_dat, trn_tgt = train.iloc[trn_idx], y.iloc[trn_idx]
    val_dat, val_tgt = train.iloc[val_idx], y.iloc[val_idx]
    
    clf = CatBoostClassifier(iterations=n_estimators, 
                                      learning_rate=0.05,
                                      depth=5,
                                      rsm = 0.7,
                                      loss_function='Logloss',
                                      eval_metric="AUC",
                                      use_best_model = True,
                                      random_strength = 20,
                                #      calc_feature_importance=True,
                                #      one_hot_max_size = 7, 
                                      l2_leaf_reg = 4,
                                      random_seed=99,
                                      od_type='Iter', 
                                      od_wait=70
                                     )
    
    # Upsample during cross validation to avoid having the same samples
    # in both train and validation sets
    # Validation set is not up-sampled to monitor overfitting
    if increase:
        # Get positive examples
        pos = pd.Series(trn_tgt == 1)
        # Add positive examples
        trn_dat = pd.concat([trn_dat, trn_dat.loc[pos]], axis=0)
        trn_tgt = pd.concat([trn_tgt, trn_tgt.loc[pos]], axis=0)
        # Shuffle data
        idx = np.arange(len(trn_dat))
        np.random.shuffle(idx)
        trn_dat = trn_dat.iloc[idx]
        trn_tgt = trn_tgt.iloc[idx]
        
    clf.fit(trn_dat, trn_tgt, 
            eval_set= (val_dat, val_tgt),
      #      cat_features =indice,
      #      eval_metric=gini_lgb,
            use_best_model=True,
            verbose=False)
    # Find best round for validation setA
    #lgb_evals[:, fold_] = clf.evals_result_["valid_1"]["gini"]
    #print(clf.evals_result_ )
    # Keep feature importances
    #imp_df[:, fold_] = clf.feature_importance_
    #Xgboost provides best round starting from 0 so it has to be incremented
    #best_round = np.argsort(lgb_evals[:, fold_])[::-1][0]

    # Predict OOF and submission probas with the best round
    oof[val_idx] = clf.predict_proba(val_dat)[:,1]
    # Update submission
    sub_preds[:, fold_] = clf.predict_proba(test)[:,1]

    # Display results
    print("Fold %2d : %.6f @%4d "
          % (fold_ + 1,
             gini_normalized(val_tgt, oof[val_idx]),
             n_estimators))
          
print("Full OOF score : %.6f" % gini_normalized(y, oof))

Fold  1 : 0.281761 @1000 
Fold  2 : 0.277750 @1000 
Fold  3 : 0.275094 @1000 
Fold  4 : 0.296755 @1000 
Fold  5 : 0.282124 @1000 
Full OOF score : 0.282477


In [18]:
print("Full OOF score : %.6f" % gini_normalized(y, oof))
oof[val_idx] = clf.predict_proba(val_dat)[:,1]
    # Update submission
sub_preds[:, fold_] = clf.predict_proba(test)[:,1]

    # Display results
print("Fold %2d : %.6f @%4d "
          % (fold_ + 1,
             gini_normalized(val_tgt, oof[val_idx]),
             n_estimators))

Full OOF score : -0.001052


CatboostError: There is no trained model to use predict(). Use fit() to train model. Then use predict().

In [38]:
import scipy
sub = pd.DataFrame()
sub['id'] = testid
sub["target"] =scipy.stats.hmean(sub_preds,axis=1)
sub.to_csv("submission_10312017_LGB_.286264_5kfold.csv", index=False, float_format="%.9f")